In [2]:
import pandas as pd
import pandas_market_calendars as mcal
import datetime
from Tools.Trade_Functions import *
import numpy as np
import plotly.express as px


/Users/atulkrishnan/Desktop/TradingAlgos/Stock_Picker_Algo


In [77]:
!ls Tools/NDX_data

NDX_2015-2014.csv           NDXdata2024.pkl
NDX_2015-2014.pkl           Nasdaq100MembersJan2015.csv
NDXdata2015.pkl             Nasdaq100MembersJan2016.csv
NDXdata2016.pkl             Nasdaq100MembersJan2017.csv
NDXdata2017.pkl             Nasdaq100MembersJan2018.csv
NDXdata2018.pkl             Nasdaq100MembersJan2019.csv
NDXdata2019.pkl             Nasdaq100MembersJan2020.csv
NDXdata2020.pkl             Nasdaq100MembersJan2021.csv
NDXdata2021.pkl             Nasdaq100MembersJan2022.csv
NDXdata2022.pkl             Nasdaq100MembersJan2023.csv
NDXdata2023.pkl             Nasdaq100MembersJan2024.csv


In [76]:
data.to_pickle("Tools/NDX_data/NDX_2015-2014.pkl")

In [3]:
longFlag =1
shortFlag =1

In [4]:
PctChange = get_SortedPctChange("2024-01-04",3,debug=False)


In [44]:
PctChange

,Ticker,PctChange
0,MRNA,0.091805
1,REGN,0.044712
2,AMGN,0.043991
3,FANG,0.032822
4,GILD,0.030490
...,...,...
96,DDOG,-0.073818
97,TEAM,-0.074876
98,AMD,-0.082016
99,MDB,-0.087734


In [45]:
PctChange[PctChange['PctChange'] >0.03]["Ticker"]

0    MRNA
1    REGN
2    AMGN
3    FANG
4    GILD
Name: Ticker, dtype: object

In [64]:
def pick_trade(TradeData,RunDate,NumStocks,stopLoss=-0.01,holdTime=0,TimeInput=5,debug=False,numDaysToCheckStopLoss=5):
    #Calculate pct change
    global longFlag
    global shortFlag
    
    
    actionDelta = 0.05
    PctChange = get_SortedPctChange(RunDate,TimeInput,debug=False)
    
    #select first N stocks and caculate percent retruns
    ShortStockList = PctChange[PctChange['PctChange'] >actionDelta]["Ticker"]
    
    LongStockList = PctChange[PctChange['PctChange'] <actionDelta]["Ticker"]
    
    longSum=0
    shortSum=0
    LongTradeInfo = None
    ShortTradeInfo = None
    
    if longFlag:
        LongTradeInfo = getLongReturns(RunDate,LongStockList,stopLoss=stopLoss,holdTime=holdTime,debug=debug,numDaysToCheckStopLoss=5)
        longSum = sum(LongTradeInfo["Returns"])

    if shortFlag:
        ShortTradeInfo = getShortReturns(RunDate,ShortStockList,stopLoss=stopLoss,holdTime=holdTime,debug=debug,numDaysToCheckStopLoss=5)
        shortSum = sum(ShortTradeInfo["Returns"])
    
    if longSum<0 and shortSum>= 0:
        longFlag = 1
        shortFlag = 1
        
    if shortSum<0 and longSum>=0:
        shortFlag =1
        longFlag = 1

    
    TotalReturn = (longSum + shortSum) / NumStocks 
    
    return TotalReturn, pd.concat([TradeData,LongTradeInfo,ShortTradeInfo])

In [65]:
framedCal= get_calendar()

In [66]:
TotalReturnList = list()
DateReturnList = list()
TimeInput = 6
#NumStocks= 2
holdTime=4
stopLoss= -1
#must be less than holdtime
numDaysToCheckStopLoss= holdTime
TradeData=pd.DataFrame()
cal=list()
for i in range(252,len(framedCal),holdTime+1):
    cal.append(framedCal[i])

#Anual Range
#year=2022
#yearlyDates = [date for date in cal if date.startswith(str(year))]
#DateRange = [yearlyDates[0], yearlyDates[-1]]

#Set custome Range
DateRange=[cal[1],cal[-1]]
#DateRange = [cal[51], cal[-1]]


DateRange=[cal.index(DateRange[0]),cal.index(DateRange[1])]
for day in cal[DateRange[0]:DateRange[1]]:
    dailyReturn,TradeData = pick_trade(TradeData,RunDate=day,NumStocks=NumStocks,stopLoss=stopLoss,TimeInput=TimeInput,holdTime=holdTime,debug=False,numDaysToCheckStopLoss=numDaysToCheckStopLoss)
    TotalReturnList.append(dailyReturn)
    
TradeData=TradeData.reset_index(drop=True)
returnDf = pd.DataFrame({'Date': pd.to_datetime(cal[DateRange[0]:DateRange[1]]),'Return':TotalReturnList})


M = 1
returnVal=[]
for i in returnDf["Return"]:
    M = M*(1+i)
    returnVal.append(M)
returnDf=pd.concat([returnDf,pd.DataFrame({"ReturnVals":returnVal})],axis=1)





In [67]:

fig = px.line(returnDf, x='Date', y='ReturnVals')

fig.update_layout(title='Returns Over Time',
                  xaxis_title='Date',
                  yaxis_title='Price')

fig.show()

In [68]:
for year in range(2015,2025):
    returnDf_year = returnDf[returnDf['Date'].dt.year == year]
    returnDf_year.loc[:, 'Return'] = returnDf_year['Return'].astype(float)
    print(year,np.prod(1 + returnDf_year['Return']))

2015 0.9681049818998608
2016 1.2050168969497028
2017 2.075370876410297
2018 0.6516794304010219
2019 2.1306659271128043
2020 2.6513496813679476
2021 2.4575542538722472
2022 0.3004535729048816
2023 2.571734247457629
2024 1.64451495125175


In [112]:
print("SharpeRatio",( np.mean(returnDf["Return"]) * 252 - 0.05) / (np.std(returnDf["Return"]) * 252**0.5))

SharpeRatio -0.15898476649452933


In [35]:
TradeData.to_csv("TradeData.csv")

In [114]:
start = getPrice("2015-01-12","AMZN","Open")
end = getPrice("2015-01-16","AMZN","Close")
print(f"Start: {start}, End: {end}, Return: {(end-start)/start}")

Start: 14.878, End: 14.537, Return: -0.02291974727785988


In [115]:
(end-start)/start -0.0004

-0.023319747277859883

In [52]:
max_drawdown = calculate_mdd(returnDf["Return"])
max_drawdown

-0.34495366057607757